# Otimização da análise do conjunto de cados "Breast Cancer Wisconsin (Diagnostic)" utilizando Algoritmo Genético

Neste notebook, exploraremos um Algoritmo Genético para otimizar o processo de classificação do câncer de mama utilizando o conjunto de dados **"Breast Cancer Wisconsin (Diagnostic) Data Set"**, disponível no [Kaggle](https://www.kaggle.com/datasets/uciml/breast-cancer-wisconsin-data/data).

---

## 📊 Visão Geral do Dataset

O **Breast Cancer Wisconsin (Diagnostic) Dataset** provém do *UCI Machine Learning Repository* e está disponível no Kaggle com o título _"breast-cancer-wisconsin-data"_.  

É amplamente utilizado em disciplinas de Ciência de Dados e *Machine Learning* como um exemplo clássico de tarefa de **classificação binária** — diagnóstico de câncer de mama **benigno (B)** ou **maligno (M)**.

---

## ℹ️ Abordagem Baseada em Algoritmo Genético

O Algoritmo Genético será utilizado para selecionar subconjuntos ideais de atributos que maximizem o desempenho de classificadores, como por exemplo o KNN.
- **O processo envolve:**
  - Geração de uma população inicial de soluções (conjuntos de atributos);
  - Avaliação da aptidão (fitness) baseada na acurácia do modelo;
  - Operadores genéticos de seleção, cruzamento e mutação;

---


## Análise exploratória dos dados

In [1]:
# Importando bibliotecas utilizadas
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
# Carregando a planilha com o dataset
dados = pd.read_csv('../../data/breast-cancer-wisconsin-data.csv') 
# visualizando cabeçalho e parte dos dados originais
dados.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


## 🧹 Remoção de Colunas Irrelevantes

Com base na documentação do dataset e na análise preliminar do cabeçalho, identificamos que as colunas `id` e `Unnamed: 32` **não possuem relevância analítica** para o nosso objetivo.

Essas colunas não apresentam **correlação com a variável-alvo `diagnosis`** (diagnóstico de tumor maligno ou benigno) e, portanto, serão removidas para:

- Eliminar ruído nos dados  
- Reduzir o risco de overfitting  
- Otimizar a performance dos modelos  
- Melhorar a interpretabilidade das análises

A limpeza de dados é uma etapa essencial para garantir resultados consistentes e confiáveis ao longo do processo analítico.

In [3]:
# Removendo colunas desnecessárias/irrevelevantes para o estudo
dados.drop(columns=['id', 'Unnamed: 32'], inplace=True)
dados.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


## 🔄 Conversão da Coluna `diagnosis`

A coluna `diagnosis` representa o diagnóstico do tumor e é uma **variável categórica**.  
Para facilitar a análise e aplicação de modelos de machine learning, realizamos a seguinte conversão para valores numéricos:

- 🟥 **Maligno (`'M'`)** → **1**
- 🟩 **Benigno (`'B'`)** → **0**

In [4]:
# Transformar a variável categória "diagnosis" em um valor numérico
dados['diagnosis'] = dados['diagnosis'].map({'M': 1, 'B': 0})
dados.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [5]:
## Descrevendo os dados para uma análise geral dos valores presentes
dados.describe()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
count,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,...,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,0.372583,14.127292,19.289649,91.969033,654.889104,0.096360,0.104341,0.088799,0.048919,0.181162,...,16.269190,25.677223,107.261213,880.583128,0.132369,0.254265,0.272188,0.114606,0.290076,0.083946
std,0.483918,3.524049,4.301036,24.298981,351.914129,0.014064,0.052813,0.079720,0.038803,0.027414,...,4.833242,6.146258,33.602542,569.356993,0.022832,0.157336,0.208624,0.065732,0.061867,0.018061
min,0.000000,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,...,7.930000,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040
25%,0.000000,11.700000,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.020310,0.161900,...,13.010000,21.080000,84.110000,515.300000,0.116600,0.147200,0.114500,0.064930,0.250400,0.071460
50%,0.000000,13.370000,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.033500,0.179200,...,14.970000,25.410000,97.660000,686.500000,0.131300,0.211900,0.226700,0.099930,0.282200,0.080040
75%,1.000000,15.780000,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.074000,0.195700,...,18.790000,29.720000,125.400000,1084.000000,0.146000,0.339100,0.382900,0.161400,0.317900,0.092080
max,1.000000,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,...,36.040000,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500



## Normalização - Padronização dos Dados
Como parte do pré-processamento, foi aplicado a padronização dos atributos numéricos, utilizando a técnicas STANDARDSCALER, que transforma os dados para que tenha média zero e desvio padrão um.

A normalização é fundamental para o bom desempenho de algoritmos que são sensíveis à escalas dos dados, como:
- KNN (K-Nearest Neighbors)

In [6]:
X = dados.drop('diagnosis', axis=1).values
y = dados['diagnosis'].values

# Normalizar
scaler = StandardScaler()
X = scaler.fit_transform(X)


## Separação dos dados de treinamento e testes

In [7]:
# Dividir em treino/teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

##  🎯 Algoritmos Genéticos

A seguir será apresentada uma implementação de algoritmo genético para otimização de hiperparâmetros do modelo de diagnóstico KNN desenvolvidos na fase 1.
A implementação contará com o desenvolvimento de uma função fitness para treinamento do modelo e operadores de seleção, cruzamento e mutação.

## 📌 Implementação do Algoritmo Genético

### 🔹 Definição de função fitness baseada nas métricas de desempenho

In [8]:
# Função fitness (usa acurácia como critério)
def fitness(individual):
    selected = np.where(individual == 1)[0]
    if len(selected) == 0:
        return 0
    clf = KNeighborsClassifier(n_neighbors=5)
    clf.fit(X_train[:, selected], y_train)
    preds = clf.predict(X_test[:, selected])
    return accuracy_score(y_test, preds)

### 🔹 Implementação operadores de seleção, cruzamento e mutação

In [9]:
# Operadores genéticos

# Parâmetros fixos
GENES = X.shape[1]

#Seleção
def selection(population, fitnesses, tournament_size):
    selected = np.zeros_like(population)
    for i in range(len(population)):
        tournament = np.random.choice(len(population), tournament_size)
        winner = tournament[np.argmax(fitnesses[tournament])]
        selected[i] = population[winner]
    return selected

#Cruzamento
def crossover(parent1, parent2):
    point = np.random.randint(1, GENES - 1)
    child1 = np.concatenate((parent1[:point], parent2[point:]))
    child2 = np.concatenate((parent2[:point], parent1[point:]))
    return child1, child2

#Mutação
def mutation(individual, mutation_rate):
    for i in range(GENES):
        if np.random.rand() < mutation_rate:
            individual[i] = 1 - individual[i]
    return individual


### 🔹 Realização de experimentos com diferentes configurações

In [10]:
# Função para gerar métricas (adaptada para incluir número de features)
def gerar_metricas(nome, y_test_metr, y_pred, num_features=None):
  return {
        'Cenário': nome,
        'Acurácia': accuracy_score(y_test_metr, y_pred),
        'Recall': recall_score(y_test_metr, y_pred),
        'Precisão': precision_score(y_test_metr, y_pred),
        'F1-Score': f1_score(y_test_metr, y_pred),
        'Número de features': num_features if num_features is not None else X.shape[1]
  }

resultados = []

# Baseline com todas features
clf_full = KNeighborsClassifier(n_neighbors=5)
clf_full.fit(X_train, y_train)
preds_full = clf_full.predict(X_test)
resultados.append(gerar_metricas("Baseline", y_test, preds_full))

# Cenários
scenarios = [
    {"name": "Cenario 1 - Padrao", "POP_SIZE":50, "GENERATIONS":100, "MUTATION_RATE":0.01, "TOURNAMENT_SIZE":5},
    {"name": "Cenario 2 - Maior Populacao, Menos Geracoes", "POP_SIZE":100, "GENERATIONS":50, "MUTATION_RATE":0.01, "TOURNAMENT_SIZE":5},
    {"name": "Cenario 3 - Maior Mutacao e Torneio", "POP_SIZE":50, "GENERATIONS":100, "MUTATION_RATE":0.05, "TOURNAMENT_SIZE":10},
]

for scenario in scenarios:
    print(f"Executando {scenario['name']}...")
    POP_SIZE = scenario["POP_SIZE"]
    GENERATIONS = scenario["GENERATIONS"]
    MUTATION_RATE = scenario["MUTATION_RATE"]
    TOURNAMENT_SIZE = scenario["TOURNAMENT_SIZE"]
    
    # Inicializar população
    population = np.random.randint(2, size=(POP_SIZE, GENES))
    
    best_fitness = 0
    best_individual = None
    
    for gen in range(GENERATIONS):
        fitnesses = np.array([fitness(ind) for ind in population])
        # print(f"Geração {gen}: Melhor fitness = {np.max(fitnesses):.4f}")
        if np.max(fitnesses) > best_fitness:
            best_fitness = np.max(fitnesses)
            best_individual = population[np.argmax(fitnesses)]
        
        selected = selection(population, fitnesses, TOURNAMENT_SIZE)
        
        new_population = []
        for i in range(0, POP_SIZE, 2):
            parent1 = selected[i]
            parent2 = selected[i+1] if i+1 < POP_SIZE else selected[0]
            child1, child2 = crossover(parent1, parent2)
            new_population.append(mutation(child1, MUTATION_RATE))
            if i+1 < POP_SIZE:
                new_population.append(mutation(child2, MUTATION_RATE))
        
        population = np.array(new_population)
    
    # Calculando métricas para o melhor indivíduo
    selected = np.where(best_individual == 1)[0]
    clf_best = KNeighborsClassifier(n_neighbors=5)
    clf_best.fit(X_train[:, selected], y_train)
    preds_best = clf_best.predict(X_test[:, selected])
    resultados.append(gerar_metricas(f"{scenario['name']}", y_test, preds_best, len(selected)))

# Criar DataFrame
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
df_resultados = pd.DataFrame(resultados)
df_resultados = df_resultados.sort_values(by='F1-Score', ascending=False)

# Exibir a tabela
print("📊 COMPARATIVO DE CENÁRIOS")
print("-" * 26)
display(df_resultados.round(4))


Executando Cenario 1 - Padrao...
Executando Cenario 2 - Maior Populacao, Menos Geracoes...
Executando Cenario 3 - Maior Mutacao e Torneio...
📊 COMPARATIVO DE CENÁRIOS
--------------------------


,Cenário,Acurácia,Recall,Precisão,F1-Score,Número de features
1,Cenario 1 - Padrao,1.0000,1.0000,1.0000,1.0000,16
3,Cenario 3 - Maior Mutacao e Torneio,1.0000,1.0000,1.0000,1.0000,15
2,"Cenario 2 - Maior Populacao, Menos Geracoes",0.9912,0.9767,1.0000,0.9882,14
0,Baseline,0.9474,0.9302,0.9302,0.9302,30


# 📊 Resultados e Interpretação dos Dados

Abordagem de seleção: Para prever o diagnóstico de câncer com base no dataset selecionado, na fase 1, utilizamos **dois modelos diferentes** com e sem normalização. Já na fase 2, foi realizada uma otimização por Algortimo Genético para seleção de features com KNN que apresentou menor desempenho na primeira fase. Ambos os experimentos foram avaliados com as seguintes métricas:

- **Acurácia**
- **Precisão**
- **Recall**
- **F1-Score**

Conforme o resultado obtido no COMPARATIVO DE MODELOS que apresenta todas as métricas para os modelos aplicados, foi possível concluir que: 

## 🔹 Análise da Fase 1 - Modelos tradicionais

- O melhor resultado foi com o modelo **Regressão Logística Normalizado**.
- Métricas obtidas: Acurária 95,61%; Recall 92,86%; Precisão: 95,12%; F1-Score: 93,98%
- Este modelo apresentou equilíbrio entre todas as métricas.
- A normalização melhorou significativamente ambos os modelos.

![alt text](image-1.png)

## 🔹 Análise da Fase 2 - Otimização com Algoritmo Genético
- O melhor resultado foi **Cenário 3 - Maior Mutação e Torneio**.
- Métricas obtidas: 100% em todas as categorias.
- Features: 16 características selecionadas
- Em relação a Baseline (sem otimização genética, 30 features) foi superior

![alt text](image-2.png)

## Observações sobre os resultados "perfeitos de 100%" da otimização
- Risco de overfitting: Resultados de 100% podem indicar overfitting no conjunto de teste;
- Tamanho da amostra de teste: o conjunto de teste talvez não tenha sido suficientemente grande para validar

## 🏆 Análise final
De acordo com as análises realizadas, a utilização de **Algoritmos Genéticos**, melhorou os resultados em relação aos modelos tradicionais conforme quadro abaixo:

Quadro comparativo entre as fases

| Aspecto | Fase 1 (Melhor) | Fase 2 (Melhor) | Diferença |
|---------|-----------------|-----------------|-----------|
| Acurácia | 95.61% | 100.00% | +4.39% |
| Recall | 92.86% | 100.00% | +7.14% |
| Precisão | 95.12% | 100.00% | +4.88% |
| F1-Score | 93.98% | 100.00% | +6.02% |

